In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd



In [3]:
data1=pd.read_excel('C:/Users/Tanmay/Desktop/Intern work/Text classification/Other factors.xlsx')
import re
def clean_text(text):
 text = re.sub(r'[^a-zA-Z\']',' ', text)
 text = re.sub(r'[^\x00-\x7F]+',' ', text)
 return text
data1['Question']=data1.Question.apply(lambda x: clean_text(x))
data1.head()

,Sr. No.,Question
0,1.0,you spoke about some challenges that you faced...
1,NaN,the quarter If you could just highlight what ...
2,NaN,currency and the external environment What ar...
3,NaN,customer specific Or could you just highlight...
4,2.0,just wanted to know are we prepared for the me...


In [4]:
data2=pd.read_excel('C:/Users/Tanmay/Desktop/Intern work/Text classification/Current status.xlsx')
data2['Question']=data2.Question.apply(lambda x: clean_text(x))
data2.head()

,Sr. No.,Question
0,1.0,what would be our nonlinear revenue as a perce...
1,2.0,last quarter we have been till last quarte...
2,NaN,been telling about aspirational growth and...
3,NaN,challenges that we found Is that something wh...
4,NaN,that long term type of aspirational growth...


In [5]:
data3=pd.read_excel('C:/Users/Tanmay/Desktop/Intern work/Text classification/Future plans.xlsx')
data3['Question']=data3.Question.apply(lambda x: clean_text(x))
data3.head()

,Sr. No.,Question
0,1.0,And do we envisage in further quarters to rema...
1,NaN,any drastic change in this year
2,2.0,Firstly we've been trying to reach out the in...
3,NaN,team is always inaccessible to us We tried ...
4,NaN,months but then we haven't got gone through...


In [6]:
from wordcloud import STOPWORDS 
cv1=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,min_df=0.02)
cv2=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,min_df=0.02)
cv3=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,min_df=0.02)
word_count_vector1=cv1.fit_transform(data1['Question'].tolist())
word_count_vector2=cv2.fit_transform(data2['Question'].tolist())
word_count_vector3=cv3.fit_transform(data3['Question'].tolist())


G:\aNAA\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [7]:
cv1=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,max_features=10000)
cv2=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,max_features=10000)
cv3=CountVectorizer(max_df=0.85,stop_words=STOPWORDS,max_features=10000)
word_count_vector1=cv1.fit_transform(data1['Question'].tolist())
word_count_vector2=cv2.fit_transform(data2['Question'].tolist())
word_count_vector3=cv3.fit_transform(data3['Question'].tolist())


In [8]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer1=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer2=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer3=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer1.fit(word_count_vector1)
tfidf_transformer2.fit(word_count_vector2)
tfidf_transformer3.fit(word_count_vector3)


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [9]:
L=len(data1['Question'])
Oth_factors=""
for i in range(L):
    Oth_factors=Oth_factors+" "+data1['Question'][i]

L=len(data2['Question'])
Cur_status=""
for i in range(L):
    Cur_status=Cur_status+" "+data2['Question'][i]

L=len(data3['Question'])
Fut_plans=""
for i in range(L):
    Fut_plans=Fut_plans+" "+data3['Question'][i]

doc=[Oth_factors,Cur_status,Fut_plans]


In [10]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=20):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results


In [13]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv3.get_feature_names()

 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer3.transform(cv3.transform([doc[2]]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 20
keywords=extract_topn_from_vector(feature_names,sorted_items,20)
 
# now print the results
print("\n=====Doc=====")
print(doc[2])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
 And do we envisage in further quarters to remain mixed    to remain more or less the same  Or we see any drastic change in this year  Firstly  we've been trying to reach out the investor relations team and your team is always inaccessible to us  We tried    we've been trying to reach out to you since last   to   months but then we haven't got    gone through  Are there any concerns  just wanted to know in the quarter    we were quite sounding bullish and also thinking of adding a lot of manpower resources and acquiring certain capabilities and all that  I think in the last quarter  a lot of things have happened globally and within India also  How do you see the trajectory of the manpower addition going ahead  Because otherwise we might be settled with deadwood  and we may not have that kind of growth that we were expecting in the business to be able to grow at say     CAGR in medium to long term  So what are your views on that  Can you just share the headcount number as